## PREPROC

In [1]:
import nibabel as nib
import os 
from fsl.wrappers import mcflirt, fslmaths, fslmerge
import subprocess

In [2]:
subj_list = [subj_dir for subj_dir in os.listdir('ds000109-2.0.2') if subj_dir[:3] == 'sub']

In [3]:
# runs : 
for subj in ['sub-01'] : # subj_list[:1]: 

    output_dir = f'ds000109-2.0.2/{subj}/preproc'
    data_dir = f'ds000109-2.0.2/{subj}'

    if not os.path.exists(output_dir) : 
        os.makedirs(output_dir)

    runs = [
        f'{subj}_task-theoryofmindwithmanualresponse_run-01_bold', 
        f'{subj}_task-theoryofmindwithmanualresponse_run-02_bold',
    ]

    for run in runs : 
        
        # Step 1: Standardization
        print(f'## Step 1 - {run} ## ')
        func_file = os.path.join(data_dir, 'func', f'{run}.nii.gz')
        standardized_output_path = os.path.join(output_dir, f'{run}_stand.nii.gz')
            
        if not os.path.exists(standardized_output_path):
            fslmaths(func_file).inm(1).run(standardized_output_path)
            print('Done')
        else : 
            print(f"Standardization already done")

        # Step 2: Motion Correction with Reference to First Volume of First Run
        print(f'## Step 2 - {run} ## ')
        mc_output_path = os.path.join(output_dir, f'{run}_stand_concat_mc.nii.gz')

        if not os.path.exists(mc_output_path):
            first_volume_reference = os.path.join(data_dir, 'func', f'{runs[0]}.nii.gz')
            mcflirt(infile=standardized_output_path, refvol=0, o=mc_output_path, plots=True, mats=True, dof=6)
            print('Done')
        else : 
            print('Motion correction already done')

        # Step 3: compute mean volume 
        print(f'## Step 3 - {run} ## ')
        mean_func = os.path.join(output_dir, f'{run}_mean.nii.gz')
        if not os.path.exists(mean_func):
            fslmaths_command = ['fslmaths', mc_output_path, '-Tmean', mean_func]
            subprocess.run(fslmaths_command)
            print('Done')
        else : print('Mean functional already done')

        # Step 4: Coreg to anatomical volum
        print(f'## Step 4 - {run} ## ')
        input_anat_image = f'ds000109-2.0.2/{subj}/anat/{subj}_T1w.nii.gz' # Your anatomical image
        output_mean_reg_T1 = os.path.join(output_dir, f'{run}_mean_func_in_T1.nii.gz')
        mat_func_to_T1 = os.path.join(output_dir, f'{run}_func_to_T1.mat')

        if not os.path.exists(output_mean_reg_T1) : 
            flirt_command = ['flirt', '-in', mean_func, '-ref', input_anat_image,
                            '-out', output_mean_reg_T1, '-omat', mat_func_to_T1,
                            '-dof', '6']
            subprocess.run(flirt_command)
            print('Done')
        else : print('Co reg already done')

        # Step 5: Reg T1 to MNI 
        print(f'## Step 5 - {run} ## ')
        reference_image_MNI = 'AAL3/MNI.nii'  # MNI template
        output_anat_image = os.path.join(output_dir, 'T1_in_MNI.nii.gz')
        mat_T1_to_MNI = os.path.join(output_dir,'T1_to_MNI.mat')   # transformation matrix output

        if not os.path.exists(output_anat_image) : 
            flirt_command = [
                'flirt', '-in', input_anat_image, '-ref', reference_image_MNI,
                '-out', output_anat_image, '-omat', mat_T1_to_MNI, '-cost', 'corratio', '-dof', '12'
            ]

            subprocess.run(flirt_command)
            print('Done')
        else : print('Reg T1 to MNI already done')

        warp_file = os.path.join(output_dir, f'T1_to_MNI_warp.nii.gz')

        if not os.path.exists(warp_file) : 
            subprocess.run([
                'fnirt', '--in=' + input_anat_image, '--aff=' + mat_T1_to_MNI,
                '--cout=' + warp_file, '--config=T1_2_MNI152_2mm'
            ])
            print('Done')
        else :
            print('warp T1-MNI already done')

        # Step 6: Apply combined transform to functional (func --> T1 --> MNI)
        print(f'## Step 6 - {run} ## ')
        func_in_mni = os.path.join(output_dir, f'{run}_func_in_MNI.nii.gz')
        if not os.path.exists(func_in_mni) : 
            applywarp_command = [
                'applywarp', '--ref=' + reference_image_MNI, '--in=' + mc_output_path,
                '--warp=' + warp_file, '--premat=' + mat_func_to_T1, '--out=' + func_in_mni
            ]
            subprocess.run(applywarp_command)
            print('Done')
        else :
            print('Apply combined transform func --> T1 --> MNI already done')

        # Step 7: Spatial Smoothing on mc file
        print(f'## Step 7 - {run} ## ')
        smoothed_output_path = os.path.join(output_dir, f'{run}_final.nii.gz')

        if not os.path.exists(smoothed_output_path):
            fslmaths(func_in_mni).s(4/2.3458).run(smoothed_output_path)
            print('Done')
        else : 
            print('Smoothing already done')

## Step 1 - sub-01_task-theoryofmindwithmanualresponse_run-01_bold ## 
Standardization already done
## Step 2 - sub-01_task-theoryofmindwithmanualresponse_run-01_bold ## 
Motion correction already done
## Step 3 - sub-01_task-theoryofmindwithmanualresponse_run-01_bold ## 
Mean functional already done
## Step 4 - sub-01_task-theoryofmindwithmanualresponse_run-01_bold ## 
Co reg already done
## Step 5 - sub-01_task-theoryofmindwithmanualresponse_run-01_bold ## 
Done
Done
## Step 6 - sub-01_task-theoryofmindwithmanualresponse_run-01_bold ## 
Done
## Step 7 - sub-01_task-theoryofmindwithmanualresponse_run-01_bold ## 
Done
## Step 1 - sub-01_task-theoryofmindwithmanualresponse_run-02_bold ## 
Done
## Step 2 - sub-01_task-theoryofmindwithmanualresponse_run-02_bold ## 
Done
## Step 3 - sub-01_task-theoryofmindwithmanualresponse_run-02_bold ## 
Done
## Step 4 - sub-01_task-theoryofmindwithmanualresponse_run-02_bold ## 
Done
## Step 5 - sub-01_task-theoryofmindwithmanualresponse_run-02_bold ## 

python(9378) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


KeyboardInterrupt: 

In [ ]:
for subj in ['sub-01'] : #subj_list: 
    for run_V1, run_V2 in zip(['run-02'], ['run2']) : #zip(['run-01', 'run-02'], ['run1', 'run2']):
        # load
        #data_path = f'ds000109-2.0.2/{subj}/func/{subj}_task-theoryofmindwithmanualresponse_{run_V1}_bold.nii.gz'
        data_path = f'ds000109-2.0.2/{subj}/preproc/sub-01_task-theoryofmindwithmanualresponse_{run_V1}_bold_func_in_MNI.nii.gz'
        
        if not os.path.isfile(data_path) : 
            continue
        img = nib.load(data_path)
        img_data = img.get_fdata()

        # set up the directory
        output_dir = f'ds000109-2.0.2/{subj}/func/{run_V2}'

        if not os.path.exists(output_dir) : 
            os.makedirs(output_dir)
            
        for slice_number in range(img_data.shape[3]) : 
            slice = img.get_fdata()[:, :, :, slice_number]
            slice_nib = nib.Nifti1Image(slice, img.affine)

            # Save
            if slice_number < 9 : 
                slice_right_number = '0000' + str(slice_number +1 )

            if (slice_number < 99) and (slice_number >= 9) :
                slice_right_number = '000' + str(slice_number + 1)
            
            if slice_number >= 99 : 
                slice_right_number = '00' + str(slice_number+1)

            file = f's8SCGorgwbua{subj}_task-theoryofmindwithmanualresponse_{run_V1}_bold_{slice_right_number}.nii'
            
            if not os.path.isfile(first_volume_reference) : 
                nib.save(slice_nib, os.path.join(output_dir, file))